In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from fractions import Fraction
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float, clean_data,reg_ex     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# parameters
customer_id = '90'
formatted_attribute = 'weight_capacity'
customer_name='%wbmason%'
buckets = """Office Chairs"""

strategy_version_input=762
attribute_id_input=2290

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
bucketz = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# _______________________________________________________________________________________________

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

# def find_first_digit(s, non=False):
#     for i, x in enumerate(s):
#         if x.isdigit() ^ non:
#             return i
#     return -1

# def split_digits(s, case=False):
#     non = True
#     while s:
#         i = find_first_digit(s, non)
#         if i == 0:
#             non = not non
#         elif i == -1:
#             yield int(s) if s.isdigit() else s if case else s.lower()
#             s = ''
#         else:
#             x, s = s[:i], s[i:]
#             yield int(x) if x.isdigit() else x if case else x.lower()

# def natural_key(s, *args, **kwargs):
#     return tuple(split_digits(s, *args, **kwargs))

# _______________________________________________________________________________________________

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
    
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
        
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))


def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

In [6]:
import json
df = query_from_file(file_name='./query/custom_fields.sql', params=params)
# df['custom_fields'] = df['custom_fields'].apply(lambda x: json.loads(x))
custom_field_df=pd.json_normalize(df['custom_fields'])
# custom_field_df = pd.json_normalize(df['custom_fields'])
fields = ['Width [Nom]', 'Width']
df_cur = pd.concat([df.drop('custom_fields', axis=1)], axis = 1)
print(len(df_cur))

898


In [7]:
# df = query_from_file(file_name='./query/uncurated.sql', params=params)
# df = df[df['bucket_id'].isin(bucketz)]
# len(df)

In [8]:
# del df['product_id']
# del df['customer_id']
# del df['attributes']
# del df['bucket_id']
# del df['values']
# df_custom=df_cur

In [9]:
# df_custom['match_cust']=df_custom['Capacity (Weight) [Nom]'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'\.00','',str(x))).apply(lambda x: re.sub(r'\.10','.1',str(x))).apply(lambda x: re.sub(r'\.20','.2',str(x))).apply(lambda x: re.sub(r'\.30','.3',str(x))).apply(lambda x: re.sub(r'\.40','.4',str(x))).apply(lambda x: re.sub(r'\.50','.5',str(x))).apply(lambda x: re.sub(r'\.60','.6',str(x))).apply(lambda x: re.sub(r'\.70','.7',str(x))).apply(lambda x: re.sub(r'\.80','.8',str(x))).apply(lambda x: re.sub(r'\.90','.9',str(x)))
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'\.290','.29',str(x))).apply(lambda x: re.sub(r'\.450','.45',str(x))).apply(lambda x: re.sub(r'lbs\.?','lb',str(x))).apply(lambda x: re.sub(r'(?<! )lb',' lb',str(x))).apply(lambda x: re.sub(r'1500','1,500',str(x)))
                                                                       
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'''\[\'|\'\]|Up to ''','',str(x)))
# df_custom[curation_col]=df_custom['match_cust']
# regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|(\d+\.\d0)|(\,)|([ac-km-rt-z])|(\.(?!\d))|(\d\d\d\d)|()'          
# df_custom['rounding'] = df_custom[curation_col].apply(lambda x: re_extract(regex_pattern_rounding, x))
# df_custom[df_custom['rounding'].astype(str)!='[]']

In [10]:
# df_custom[curation_col]=df_custom[curation_col].apply(lambda x: str(f'["{x}"]'))
# matchcustom=df_custom[['external_id',curation_col]]
# print(len(matchcustom))
# matchcustom

In [11]:
regex_pattern=r'(?i)(suitable\s?.{0,25}\d+\s?lb)|(support.{0,15}weighi?n?g?\s?\d+\s?lb)|(support.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static load capacity)|((?<!\d)(?<!\.)\d+\s?lb.?.?\s?capacity)|(store.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static\s?weight\s?capacity)|(users\s?up\s?to\s\d+\s?lb)|(\d+\s?lb.?.?\s?max\s?capacity)|(\d+\s?lb.?.?\s?seating\s?capacity)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(supports\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)|(\d+\s?lb.?.?\s?Load\s?Capacity)|(weight capacity.?o?f?.?.?\s?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?weight capacity)|([^weighs]\s\d+.?\s?lb.?\s?weight capacity)|(\s\d*\,\d*\s?lb.?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?weight capacity)|(\s(?<!\d)(?<!\.)\d+\.\d+\s?lb.?\s?weight capacity)|(\s?(?<!\d)(?<!\.)\d*.?\d*\s?lb.?\s?d?i?s?t?r?i?b?u?t?e?d?\s?weight capacity)|(weight\s?capacity\s?of\s?\d+\s?lb)|(\s?\d+.?\d*\s?p?o?u?n?d?\s?weight capacity)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(m?a?x?i?m?u?m?\s?.{0,15}weight capacity.?\s?\d*.?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\s?\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(\s\d*\,\d*\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(m?a?x?i?m?u?m?\s?weight\s?capacity.?\s?\(?s?e?a?t?i?n?g?\)?\s?.?\s?\d*\s?lb)|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(maxi?m?u?m?\s?.{0,15}w?e?i?g?h?t?.?\s?\d+\s?lb)|(seats.{0,15}\d+\s?lb(?!\s?and.?up))|((?<!weighs.)\d+.?\d*.?l?b?\s?to\s?\d+.?\d*\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?-\s?\d+.?\d*\s?lb)|(up.?to.?\d+\.?\d*.?lb)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(\d\.?\d*\s?lb\W{0,3}Cap\b)|()'  
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

regex_pattern_blank=r'\d'
df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_newest=df_cur[df_cur['new_matches'].astype(str)!='[]']

regex_lbs=r"(?i)(\d+\.?\d*\s?lb)|()"
df_newest['newest_matches']=df_newest['matches'].apply(lambda x: re_extract(regex_lbs, str(x)))

df_newest=df_newest[df_newest['newest_matches'].astype(str)!='[]']

df_newest[curation_col]=df_newest['newest_matches']
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'''12.0 in''','12 in',str(x)))
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'''\.125''','.13',str(x)))
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'''3.12 in","3.13 in''','3.12 in',str(x)))
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'''4.12 in","4.13 in''','4.12 in',str(x)))
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'''\.875''','.88',str(x)))
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'''\.625''','.63 in',str(x)))
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'\.00','',str(x))).apply(lambda x: re.sub(r'\.10','.1',str(x))).apply(lambda x: re.sub(r'\.20','.2',str(x))).apply(lambda x: re.sub(r'\.30','.3',str(x))).apply(lambda x: re.sub(r'\.40','.4',str(x))).apply(lambda x: re.sub(r'\.50','.5',str(x))).apply(lambda x: re.sub(r'\.60','.6',str(x))).apply(lambda x: re.sub(r'\.70','.7',str(x))).apply(lambda x: re.sub(r'\.80','.8',str(x))).apply(lambda x: re.sub(r'\.90','.9',str(x)))  
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'''6000''','6,000',str(x)))
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'''1200''','1,200',str(x)))
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'''5000''','5,000',str(x)))
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'''(?<!\s)lb''',' lb',str(x))).apply(lambda x: re.sub(r"\'",'"',str(x))).apply(lambda x: re.sub(r'''" | "''','"',str(x)))

regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|(\d+\.\d0)|(\d{4})|(\d\d\d\d)|()'         
df_newest['rounding'] = df_newest[curation_col].apply(lambda x: re_extract(regex_pattern_rounding, x))
df_newest[df_newest['rounding'].astype(str)!='[]']

Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, matches, new_matches, newest_matches, Q:weight_capacity, rounding]
Index: []

In [12]:
df_newest[df_newest['external_id'].astype(str)=='5637553386']

Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, matches, new_matches, newest_matches, Q:weight_capacity, rounding]
Index: []

In [13]:
matches_natural=df_newest[['external_id',curation_col]]
print(len(matches_natural))
matches_natural

77


external_id    Q:weight_capacity
208  5637588170           ["200 lb"]
213  5637591040           ["200 lb"]
223  5637592580           ["110 lb"]
224  5637592664           ["200 lb"]
225  5637592650           ["200 lb"]
226  5637592691           ["150 lb"]
227  5637592692           ["150 lb"]
229  5637592686           ["400 lb"]
230  5637592689           ["150 lb"]
236  5637605087           ["332 lb"]
237  5637605088           ["332 lb"]
238  5637605089           ["100 lb"]
251  5637605086           ["660 lb"]
346  5637605308          ["17.6 lb"]
362  5637453458           ["300 lb"]
363  5637453460           ["400 lb"]
364  5637453461           ["300 lb"]
365  5637453462           ["300 lb"]
366  5637502219           ["250 lb"]
369  5637566217           ["250 lb"]
470  5637605047           ["300 lb"]
471  5637605048           ["300 lb"]
472  5637605067           ["300 lb"]
482  5637605057           ["300 lb"]
483  5637605058           ["300 lb"]
485  5637605037           ["300 lb"]
486  5637605038           ["300 lb"]
489  5637605041           ["300 lb"]
491  5637605043           ["300 lb"]
492  5637605044           ["300 lb"]
493  5637605045           ["300 lb"]
494  5637605049           ["300 lb"]
496  5637605051           ["300 lb"]
497  5637605052           ["300 lb"]
498  5637605053           ["300 lb"]
503  5637605061           ["300 lb"]
504  5637605062           ["300 lb"]
507  5637605065           ["300 lb"]
508  5637485633           ["250 lb"]
510  5637605033           ["300 lb"]
511  5637605034           ["300 lb"]
695  5637591142            ["40 lb"]
696  5637591143            ["40 lb"]
697  5637591144            ["45 lb"]
698  5637591145            ["45 lb"]
699  5637591146            ["40 lb"]
700  5637591138            ["40 lb"]
701  5637591139            ["40 lb"]
702  5637591140            ["40 lb"]
703  5637591141            ["40 lb"]
704  5637591147            ["40 lb"]
782  5637605208  ["500 lb","300 lb"]
795  5637605200           ["500 lb"]
796  5637605201           ["500 lb"]
797  5637605202  ["500 lb","300 lb"]
798  5637605203  ["500 lb","300 lb"]
799  5637605204  ["500 lb","300 lb"]
800  5637605205  ["500 lb","300 lb"]
801  5637605206  ["500 lb","300 lb"]
802  5637605207  ["500 lb","300 lb"]
803  5637605217  ["500 lb","300 lb"]
804  5637605218  ["500 lb","300 lb"]
805  5637605219  ["500 lb","300 lb"]
806  5637605220  ["500 lb","300 lb"]
812  5637605216  ["500 lb","300 lb"]
813  5637605221  ["500 lb","300 lb"]
814  5637605222  ["500 lb","300 lb"]
830  5637605197           ["500 lb"]
831  5637605198           ["500 lb"]
832  5637605199           ["500 lb"]
872  5637605096           ["500 lb"]
875  5637605091           ["500 lb"]
876  5637605092           ["500 lb"]
879  5637605095           ["500 lb"]
894  5637603519   ["50 lb","100 lb"]
895  5637603520           ["250 lb"]
896  5637603521           ["250 lb"]

In [14]:
# df_newest

# N/A

In [15]:
regex_pattern_na=r'(?i)(up.?to.?\d+\s?lb)|(suitable\s?.{0,25}\d+\s?lb)|(support.{0,15}weighi?n?g?\s?\d+\s?lb)|(support.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static load capacity)|(\d+\s?lb.?.?\s?capacity)|(store.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static\s?weight\s?capacity)|(users\s?up\s?to\s\d+\s?lb)|(\d+\s?lb.?.?\s?max\s?capacity)|(\d+\s?lb.?.?\s?seating\s?capacity)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(supports\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)|(\d+\s?lb.?.?\s?Load\s?Capacity)|(weight capacity.?o?f?.?.?\s?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?weight capacity)|([^weighs]\s\d+.?\s?lb.?\s?weight capacity)|(\s\d*\,\d*\s?lb.?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?weight capacity)|(\s?\d*.?\d*\s?lb.?\s?d?i?s?t?r?i?b?u?t?e?d?\s?weight capacity)|(weight\s?capacity\s?of\s?\d+\s?lb)|(\s?\d+.?\d*\s?p?o?u?n?d?\s?weight capacity)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(m?a?x?i?m?u?m?\s?.{0,15}weight capacity.?\s?\d*.?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\s?\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(\s\d*\,\d*\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(m?a?x?i?m?u?m?\s?weight\s?capacity.?\s?\(?s?e?a?t?i?n?g?\)?\s?.?\s?\d*\s?lb)|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(maxi?m?u?m?\s?.{0,15}w?e?i?g?h?t?.?\s?\d+\s?lb)|(Mini?m?u?m?.{0,15}\s?w?e?i?g?h?t?.?\s?\d+\s?lb)|(seats.{0,15}\d+\s?lb)'   
df_cur['matches_na'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df_na=df_cur[(df_cur['matches_na'].astype(str)=='[]')]#&(df['matches_lb'].astype(str)!='[]')] 

In [16]:
regex_pattern_na=r'(?i)(\d[^\w]?lb)'   
df_na['double_check'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
print(len(df_na[df_na['double_check'].astype(str)=='[]']))
na_final=df_na[df_na['double_check'].astype(str)=='[]']

794


In [17]:
na_final[curation_col]='n/a'
match_na=na_final[['external_id',curation_col]]
match_na

external_id Q:weight_capacity
0    5637532942               n/a
1    5637604520               n/a
2    5637603361               n/a
3    5637564422               n/a
4    5637564423               n/a
..          ...               ...
890  5637590290               n/a
891  5637590291               n/a
892  5637594491               n/a
893  5637603224               n/a
897  5637590189               n/a

[794 rows x 2 columns]

In [18]:
# na_final[0:500]

In [19]:
# print(len(df))
# print(len(match_na)+len(matches_natural)+len(matchcustom))

In [20]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(matches_natural))
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(match_na))
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matches_natural))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matches_width))

matches_natural
match_na


NameError: name 'matchcustom' is not defined

In [ ]:
stop

# send to the folder for upload

In [25]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].vpandastype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matches_natural)

In [26]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, df, match_na)

In [24]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name

# def looks_good(customer, attribute, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

# looks_good('WBMason', attribute, df, matchcustom)